# Example of `InferenceData` schema in PyMC3
The description of the `InferenceData` structure can be found [here](schema.html).

In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,25200,12.0
Erika,27500,18.0


In [3]:
with pm.Model() as model:
    time_since_joined = pm.Data("time_since_joined", time)
    
    sigma = pm.HalfNormal('sigma', sd=30)
    b0 = pm.Normal("b0", mu=0, sd=10)
    b1 = pm.Normal("b1", mu=0, sd=10)
    
    pm.Normal("slack_comments", mu=b0 + b1 * time_since_joined, sigma=sigma, observed=slack_comments)
    
    trace = pm.sample(200, chains=4)
    posterior_predictive = pm.sample_posterior_predictive(trace)
    prior = pm.sample_prior_predictive(150)

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b1, b0, sigma]
Sampling 4 chains, 0 divergences: 100%|██████████| 2800/2800 [00:03<00:00, 837.12draws/s] 
The acceptance probability does not match the target. It is 0.9005350539239327, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9627238596322175, but should be close to 0.8. Try to increase the number of tuning steps.
100%|██████████| 800/800 [00:00<00:00, 1253.93it/s]


In [4]:
idata_pymc3 = az.from_pymc3(
    trace,
    prior=prior,
    posterior_predictive=posterior_predictive,
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "log_likelihood": ["developer"],
        "time_since_joined": ["developer"],
    }
)

In [5]:
idata_pymc3

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data
	> constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [6]:
idata_pymc3.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 200)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 192 193 194 195 196 197 198 199
Data variables:
    b0       (chain, draw) float64 -3.04 15.18 0.7325 ... 0.744 13.07 -6.569
    b1       (chain, draw) float64 99.58 62.15 92.96 67.5 ... 81.02 71.53 92.44
    sigma    (chain, draw) float64 1.11e+03 1.126e+03 ... 1.105e+03 1.13e+03
Attributes:
    created_at:                 2019-11-17T20:56:09.931747
    inference_library:          pymc3
    inference_library_version:  3.7

In [7]:
idata_pymc3.sample_stats

<xarray.Dataset>
Dimensions:           (chain: 4, developer: 5, draw: 200)
Coordinates:
  * chain             (chain) int64 0 1 2 3
  * draw              (draw) int64 0 1 2 3 4 5 6 ... 193 194 195 196 197 198 199
  * developer         (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    energy_error      (chain, draw) float64 0.5124 -0.2888 ... 0.07027 0.1273
    mean_tree_accept  (chain, draw) float64 0.5318 1.0 0.9727 ... 0.9433 0.8357
    step_size_bar     (chain, draw) float64 0.9268 0.9268 ... 0.6053 0.6053
    max_energy_error  (chain, draw) float64 0.9052 -0.299 ... 0.1105 0.3647
    step_size         (chain, draw) float64 0.813 0.813 0.813 ... 1.129 1.129
    depth             (chain, draw) int64 2 2 2 2 2 2 2 1 2 ... 3 3 1 3 3 3 3 3
    diverging         (chain, draw) bool False False False ... False False False
    lp                (chain, draw) float64 -1.466e+03 -1.465e+03 ... -1.466e+03
    energy            (chain, draw) float64 1.468e+03 1.467e+03 ... 1.469e+03
    tune              (chain, draw) bool True False False ... False False False
    tree_size         (chain, draw) float64 3.0 3.0 3.0 3.0 ... 7.0 7.0 7.0 7.0
    log_likelihood    (chain, draw, developer) float64 -28.14 -44.61 ... -269.6
Attributes:
    created_at:                 2019-11-17T20:56:10.018130
    inference_library:          pymc3
    inference_library_version:  3.7

In [8]:
idata_pymc3.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 200)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 1.823e+03 703.1 ... 356.9
Attributes:
    created_at:                 2019-11-17T20:56:10.021047
    inference_library:          pymc3
    inference_library_version:  3.7

In [9]:
idata_pymc3.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) float64 7.5e+03 1.01e+04 ... 2.52e+04 2.75e+04
Attributes:
    created_at:                 2019-11-17T20:56:10.024234
    inference_library:          pymc3
    inference_library_version:  3.7

In [10]:
idata_pymc3.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2019-11-17T20:56:10.024877
    inference_library:          pymc3
    inference_library_version:  3.7

In [11]:
idata_pymc3.prior

<xarray.Dataset>
Dimensions:         (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain           (chain) int64 0
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 5.228 87.69 ... 29.04 23.6
    b0              (chain, draw) float64 -12.91 -2.018 4.764 ... 1.053 -15.27
    b1              (chain, draw) float64 3.752 8.683 -2.673 ... 1.052 2.671
    sigma           (chain, draw) float64 32.27 16.44 17.12 ... 29.65 17.65
    sigma_log__     (chain, draw) float64 3.474 2.8 2.84 ... 1.872 3.389 2.871
Attributes:
    created_at:                 2019-11-17T20:56:10.022503
    inference_library:          pymc3
    inference_library_version:  3.7